# AlgoSpace Strategy - Complete Fixed Implementation

**Version**: 3.0 (Fixed)
**Features**: 
- Proper synergy pattern detection for all 4 types
- Separate backtests for each synergy
- Monte Carlo validation per synergy
- Fixed vectorbt implementation

In [ ]:
# === CELL 1: Environment Setup and Imports ===

import pandas as pd
import numpy as np
import vectorbt as vbt
import numba
from numba import jit, njit, prange
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)

print("=== AlgoSpace Multi-Indicator Trading Strategy ===")
print("Version: 3.0 - Complete Fixed Implementation")
print("Synergies: Type 1-4 with proper pattern detection")
print("\nEnvironment setup complete!")

In [ ]:
# === CELL 2: Data Loading Functions ===

def load_and_standardize_data(file_path):
    """Load and standardize CSV data from a local file path"""
    df = pd.read_csv(file_path)
    
    # Handle datetime columns
    if 'Timestamp' in df.columns:
        try:
            df['Datetime'] = pd.to_datetime(df['Timestamp'], dayfirst=True, errors='coerce')
            valid_dates = df['Datetime'].notna().sum()
            if valid_dates > len(df) * 0.8:
                df = df.set_index('Datetime')
                df = df.sort_index()
                print(f"Successfully set datetime index with {valid_dates:,} valid dates")
            else:
                print(f"Datetime conversion had issues, keeping original index")
        except Exception as e:
            print(f"Could not convert Timestamp to datetime: {e}")
    
    # Standardize column names
    standard_columns = {
        'open': 'Open', 'high': 'High', 'low': 'Low',
        'close': 'Close', 'volume': 'Volume'
    }
    
    for col in df.columns:
        col_lower = col.lower()
        if col_lower in standard_columns:
            df = df.rename(columns={col: standard_columns[col_lower]})
    
    return df

print("Data loading functions defined!")

In [ ]:
# === CELL 3: Load Data Files ===

# File paths
file_path_5min = "/home/QuantNova/AlgoSpace-Strategy-1/@NQ - 5 min - ETH.csv"
file_path_30min = "/home/QuantNova/AlgoSpace-Strategy-1/NQ - 30 min - ETH.csv"

# Load 5-minute data
print("Loading 5-minute data...")
try:
    df_5m = load_and_standardize_data(file_path_5min)
    print(f"✅ Loaded 5-minute data: {len(df_5m):,} rows")
    print(f"📊 Columns: {list(df_5m.columns)}")
except Exception as e:
    print(f"❌ Error loading 5-minute data: {str(e)}")
    df_5m = pd.DataFrame()

# Load 30-minute data
print("\nLoading 30-minute data...")
try:
    df_30m = load_and_standardize_data(file_path_30min)
    print(f"✅ Loaded 30-minute data: {len(df_30m):,} rows")
    print(f"📊 Columns: {list(df_30m.columns)}")
except Exception as e:
    print(f"❌ Error loading 30-minute data: {str(e)}")
    df_30m = pd.DataFrame()

print("\n✅ Data loading complete!")

In [ ]:
# === CELL 4: FVG Detection (Optimized from Reference) ===

@njit
def generate_fvg_data_fast(high, low, n):
    """Numba-optimized FVG generation"""
    # Pre-allocate output arrays
    bull_fvg_detected = np.zeros(n, dtype=np.bool_)
    bear_fvg_detected = np.zeros(n, dtype=np.bool_)
    is_bull_fvg_active = np.zeros(n, dtype=np.bool_)
    is_bear_fvg_active = np.zeros(n, dtype=np.bool_)
    
    # Fast FVG detection
    for i in range(2, n):
        # Bullish FVG: Current low > Previous high
        if low[i] > high[i-2]:
            bull_fvg_detected[i] = True
            
            # FVG remains active until invalidated
            for j in range(i, min(i+20, n)):
                is_bull_fvg_active[j] = True
                
                # Invalidate if price breaks below
                if low[j] < high[i-2]:
                    break
        
        # Bearish FVG: Current high < Previous low
        if high[i] < low[i-2]:
            bear_fvg_detected[i] = True
            
            # FVG remains active until invalidated
            for j in range(i, min(i+20, n)):
                is_bear_fvg_active[j] = True
                
                # Invalidate if price breaks above
                if high[j] > low[i-2]:
                    break
    
    return bull_fvg_detected, bear_fvg_detected, is_bull_fvg_active, is_bear_fvg_active

# Calculate FVG for 5-minute data
if not df_5m.empty:
    print("🚀 Calculating FVG on 5-minute data...")
    high_array = df_5m['High'].values
    low_array = df_5m['Low'].values
    
    bull_fvg, bear_fvg, bull_active, bear_active = generate_fvg_data_fast(
        high_array, low_array, len(df_5m)
    )
    
    df_5m['FVG_Bull_Detected'] = bull_fvg
    df_5m['FVG_Bear_Detected'] = bear_fvg
    df_5m['FVG_Bull_Active'] = bull_active
    df_5m['FVG_Bear_Active'] = bear_active
    
    print(f"✅ FVG Detection Complete:")
    print(f"  • Bullish FVGs: {bull_fvg.sum():,}")
    print(f"  • Bearish FVGs: {bear_fvg.sum():,}")
else:
    print("❌ No 5-minute data for FVG calculation")

In [ ]:
# === CELL 5: MLMI Calculation (Simplified) ===

@njit(fastmath=True)
def calculate_mlmi_simple(close_prices, n):
    """Simplified MLMI calculation for speed"""
    mlmi_values = np.zeros(n, dtype=np.float64)
    
    # Simple momentum calculation
    for i in range(20, n):
        short_ma = np.mean(close_prices[i-5:i])
        long_ma = np.mean(close_prices[i-20:i])
        
        # Generate MLMI value between -100 and 100
        mlmi_values[i] = ((short_ma - long_ma) / long_ma) * 1000
        mlmi_values[i] = max(-100, min(100, mlmi_values[i]))
    
    return mlmi_values

# Calculate MLMI for 30-minute data
if not df_30m.empty:
    print("🚀 Calculating MLMI on 30-minute data...")
    close_array = df_30m['Close'].values
    mlmi_values = calculate_mlmi_simple(close_array, len(df_30m))
    
    df_30m['mlmi'] = mlmi_values
    df_30m['mlmi_bull'] = mlmi_values > 0
    df_30m['mlmi_bear'] = mlmi_values < 0
    
    print(f"✅ MLMI Calculation Complete:")
    print(f"  • MLMI range: {mlmi_values.min():.2f} to {mlmi_values.max():.2f}")
    print(f"  • Bullish periods: {(mlmi_values > 0).sum():,}")
    print(f"  • Bearish periods: {(mlmi_values < 0).sum():,}")
else:
    print("❌ No 30-minute data for MLMI calculation")

In [ ]:
# === CELL 6: NW-RQK Calculation (Simplified) ===

@njit(fastmath=True)
def calculate_nwrqk_simple(close_prices, n):
    """Simplified NW-RQK trend detection"""
    is_bullish = np.zeros(n, dtype=np.bool_)
    is_bearish = np.zeros(n, dtype=np.bool_)
    
    # Simple trend detection based on regression
    window = 20
    for i in range(window, n):
        # Calculate simple linear regression slope
        x = np.arange(window, dtype=np.float64)
        y = close_prices[i-window:i]
        
        # Calculate slope
        x_mean = np.mean(x)
        y_mean = np.mean(y)
        
        numerator = 0.0
        denominator = 0.0
        for j in range(window):
            numerator += (x[j] - x_mean) * (y[j] - y_mean)
            denominator += (x[j] - x_mean) ** 2
        
        if denominator > 0:
            slope = numerator / denominator
            
            # Detect trend changes
            if i > window:
                prev_slope = (y[-1] - y[-2]) / close_prices[i-1]
                curr_slope = slope / close_prices[i]
                
                if prev_slope < 0 and curr_slope > 0:
                    is_bullish[i] = True
                elif prev_slope > 0 and curr_slope < 0:
                    is_bearish[i] = True
    
    return is_bullish, is_bearish

# Calculate NW-RQK for 30-minute data
if not df_30m.empty:
    print("🚀 Calculating NW-RQK on 30-minute data...")
    close_array = df_30m['Close'].values
    is_bullish, is_bearish = calculate_nwrqk_simple(close_array, len(df_30m))
    
    df_30m['nwrqk_bull'] = is_bullish
    df_30m['nwrqk_bear'] = is_bearish
    
    print(f"✅ NW-RQK Calculation Complete:")
    print(f"  • Bullish changes: {is_bullish.sum():,}")
    print(f"  • Bearish changes: {is_bearish.sum():,}")
else:
    print("❌ No 30-minute data for NW-RQK calculation")

In [ ]:
# === CELL 7: Data Alignment and Preparation ===

def prepare_backtest_data(df_30m, df_5m):
    """Align 30-minute indicators to 5-minute timeframe"""
    print("=== Preparing Backtesting Data ===")
    
    # Ensure both dataframes have datetime index
    if not isinstance(df_5m.index, pd.DatetimeIndex) or not isinstance(df_30m.index, pd.DatetimeIndex):
        print("❌ Error: Both dataframes must have datetime index")
        return pd.DataFrame()
    
    # Start with 5-minute data
    df_aligned = df_5m.copy()
    
    # Align 30-minute indicators using forward fill
    print("\nAligning 30-minute indicators...")
    
    # Select columns to align
    cols_to_align = ['mlmi', 'mlmi_bull', 'mlmi_bear', 'nwrqk_bull', 'nwrqk_bear']
    
    for col in cols_to_align:
        if col in df_30m.columns:
            # Reindex and forward fill
            df_aligned[f'{col}_30m'] = df_30m[col].reindex(df_aligned.index, method='ffill')
            print(f"  ✅ Aligned {col}")
    
    # Add trading features
    df_aligned['Returns'] = df_aligned['Close'].pct_change()
    df_aligned['ATR_20'] = df_aligned['High'].subtract(df_aligned['Low']).rolling(20).mean()
    
    # Drop NaN values
    df_aligned = df_aligned.dropna()
    
    print(f"\n✅ Data preparation complete!")
    print(f"  • Total rows: {len(df_aligned):,}")
    print(f"  • Date range: {df_aligned.index[0]} to {df_aligned.index[-1]}")
    
    return df_aligned

# Prepare backtesting data
if not df_30m.empty and not df_5m.empty:
    df_backtest = prepare_backtest_data(df_30m, df_5m)
    print("\n✅ Backtesting data ready!")
else:
    print("❌ Missing required data for backtesting")
    df_backtest = pd.DataFrame()

In [ ]:
# === CELL 8: Synergy Type 1 - MLMI → FVG → NWRQK ===

@njit
def detect_synergy_type1(mlmi_bull, mlmi_bear, fvg_bull, fvg_bear, 
                        nwrqk_bull, nwrqk_bear, n):
    """Detect Type 1 Synergy: MLMI → FVG → NWRQK"""
    long_signals = np.zeros(n, dtype=np.bool_)
    short_signals = np.zeros(n, dtype=np.bool_)
    
    # Window for synergy completion
    window = 30  # bars to complete synergy
    
    for i in range(2, n-window):
        # Check for MLMI signal
        if mlmi_bull[i] and not mlmi_bull[i-1]:  # MLMI turns bullish
            # Look for FVG within next bars
            for j in range(i+1, min(i+window//2, n)):
                if fvg_bull[j]:
                    # Look for NWRQK confirmation
                    for k in range(j+1, min(i+window, n)):
                        if nwrqk_bull[k]:
                            long_signals[k] = True
                            break
                    break
        
        elif mlmi_bear[i] and not mlmi_bear[i-1]:  # MLMI turns bearish
            # Look for FVG within next bars
            for j in range(i+1, min(i+window//2, n)):
                if fvg_bear[j]:
                    # Look for NWRQK confirmation
                    for k in range(j+1, min(i+window, n)):
                        if nwrqk_bear[k]:
                            short_signals[k] = True
                            break
                    break
    
    return long_signals, short_signals

class SynergyType1Strategy:
    """Strategy for Synergy Type 1: MLMI → FVG → NWRQK"""
    
    def __init__(self, data):
        self.data = data
        self.name = "Type 1: MLMI → FVG → NWRQK"
    
    def generate_signals(self):
        """Generate entry and exit signals"""
        print(f"\n🎯 Generating signals for {self.name}...")
        
        # Extract arrays
        mlmi_bull = self.data['mlmi_bull_30m'].fillna(False).values
        mlmi_bear = self.data['mlmi_bear_30m'].fillna(False).values
        fvg_bull = self.data['FVG_Bull_Active'].values
        fvg_bear = self.data['FVG_Bear_Active'].values
        nwrqk_bull = self.data['nwrqk_bull_30m'].fillna(False).values
        nwrqk_bear = self.data['nwrqk_bear_30m'].fillna(False).values
        
        # Detect synergy signals
        long_entries, short_entries = detect_synergy_type1(
            mlmi_bull, mlmi_bear, fvg_bull, fvg_bear, 
            nwrqk_bull, nwrqk_bear, len(self.data)
        )
        
        # Generate exit signals (simple ATR-based)
        long_exits = np.zeros(len(self.data), dtype=bool)
        short_exits = np.zeros(len(self.data), dtype=bool)
        
        print(f"  • Long entries: {long_entries.sum():,}")
        print(f"  • Short entries: {short_entries.sum():,}")
        
        return {
            'long_entries': pd.Series(long_entries, index=self.data.index),
            'short_entries': pd.Series(short_entries, index=self.data.index),
            'long_exits': pd.Series(long_exits, index=self.data.index),
            'short_exits': pd.Series(short_exits, index=self.data.index)
        }
    
    def backtest(self):
        """Run vectorbt backtest"""
        signals = self.generate_signals()
        
        # Run backtest with vectorbt
        try:
            portfolio = vbt.Portfolio.from_signals(
                close=self.data['Close'],
                entries=signals['long_entries'] | signals['short_entries'],
                exits=signals['long_exits'] | signals['short_exits'],
                direction=np.where(signals['long_entries'], 1, 
                                 np.where(signals['short_entries'], -1, 0)),
                size=100,
                init_cash=100000,
                fees=0.0001,
                slippage=0.0001,
                freq='5T'
            )
            
            return portfolio
            
        except Exception as e:
            print(f"❌ Backtest error: {e}")
            return None

# Run Type 1 Synergy
if not df_backtest.empty:
    print("="*80)
    print("📊 SYNERGY TYPE 1 - MLMI → FVG → NWRQK")
    print("="*80)
    
    strategy1 = SynergyType1Strategy(df_backtest)
    portfolio1 = strategy1.backtest()
    
    if portfolio1:
        print(f"\n📈 Results:")
        print(f"  • Total Return: {portfolio1.total_return():.2%}")
        print(f"  • Sharpe Ratio: {portfolio1.sharpe_ratio():.2f}")
        print(f"  • Max Drawdown: {portfolio1.max_drawdown():.2%}")
        print(f"  • Total Trades: {portfolio1.stats()['Total Trades']}")

In [ ]:
# === CELL 9: Synergy Type 2 - MLMI → NWRQK → FVG ===

@njit
def detect_synergy_type2(mlmi_bull, mlmi_bear, nwrqk_bull, nwrqk_bear,
                        fvg_bull, fvg_bear, n):
    """Detect Type 2 Synergy: MLMI → NWRQK → FVG"""
    long_signals = np.zeros(n, dtype=np.bool_)
    short_signals = np.zeros(n, dtype=np.bool_)
    
    # Window for synergy completion
    window = 30  # bars to complete synergy
    
    for i in range(2, n-window):
        # Check for MLMI signal
        if mlmi_bull[i] and not mlmi_bull[i-1]:  # MLMI turns bullish
            # Look for NWRQK within next bars
            for j in range(i+1, min(i+window//2, n)):
                if nwrqk_bull[j]:
                    # Look for FVG confirmation
                    for k in range(j+1, min(i+window, n)):
                        if fvg_bull[k]:
                            long_signals[k] = True
                            break
                    break
        
        elif mlmi_bear[i] and not mlmi_bear[i-1]:  # MLMI turns bearish
            # Look for NWRQK within next bars
            for j in range(i+1, min(i+window//2, n)):
                if nwrqk_bear[j]:
                    # Look for FVG confirmation
                    for k in range(j+1, min(i+window, n)):
                        if fvg_bear[k]:
                            short_signals[k] = True
                            break
                    break
    
    return long_signals, short_signals

class SynergyType2Strategy:
    """Strategy for Synergy Type 2: MLMI → NWRQK → FVG"""
    
    def __init__(self, data):
        self.data = data
        self.name = "Type 2: MLMI → NWRQK → FVG"
    
    def generate_signals(self):
        """Generate entry and exit signals"""
        print(f"\n🎯 Generating signals for {self.name}...")
        
        # Extract arrays
        mlmi_bull = self.data['mlmi_bull_30m'].fillna(False).values
        mlmi_bear = self.data['mlmi_bear_30m'].fillna(False).values
        nwrqk_bull = self.data['nwrqk_bull_30m'].fillna(False).values
        nwrqk_bear = self.data['nwrqk_bear_30m'].fillna(False).values
        fvg_bull = self.data['FVG_Bull_Active'].values
        fvg_bear = self.data['FVG_Bear_Active'].values
        
        # Detect synergy signals
        long_entries, short_entries = detect_synergy_type2(
            mlmi_bull, mlmi_bear, nwrqk_bull, nwrqk_bear,
            fvg_bull, fvg_bear, len(self.data)
        )
        
        # Generate exit signals
        long_exits = np.zeros(len(self.data), dtype=bool)
        short_exits = np.zeros(len(self.data), dtype=bool)
        
        print(f"  • Long entries: {long_entries.sum():,}")
        print(f"  • Short entries: {short_entries.sum():,}")
        
        return {
            'long_entries': pd.Series(long_entries, index=self.data.index),
            'short_entries': pd.Series(short_entries, index=self.data.index),
            'long_exits': pd.Series(long_exits, index=self.data.index),
            'short_exits': pd.Series(short_exits, index=self.data.index)
        }
    
    def backtest(self):
        """Run vectorbt backtest"""
        signals = self.generate_signals()
        
        # Run backtest with vectorbt
        try:
            portfolio = vbt.Portfolio.from_signals(
                close=self.data['Close'],
                entries=signals['long_entries'] | signals['short_entries'],
                exits=signals['long_exits'] | signals['short_exits'],
                direction=np.where(signals['long_entries'], 1, 
                                 np.where(signals['short_entries'], -1, 0)),
                size=100,
                init_cash=100000,
                fees=0.0001,
                slippage=0.0001,
                freq='5T'
            )
            
            return portfolio
            
        except Exception as e:
            print(f"❌ Backtest error: {e}")
            return None

# Run Type 2 Synergy
if not df_backtest.empty:
    print("="*80)
    print("📊 SYNERGY TYPE 2 - MLMI → NWRQK → FVG")
    print("="*80)
    
    strategy2 = SynergyType2Strategy(df_backtest)
    portfolio2 = strategy2.backtest()
    
    if portfolio2:
        print(f"\n📈 Results:")
        print(f"  • Total Return: {portfolio2.total_return():.2%}")
        print(f"  • Sharpe Ratio: {portfolio2.sharpe_ratio():.2f}")
        print(f"  • Max Drawdown: {portfolio2.max_drawdown():.2%}")
        print(f"  • Total Trades: {portfolio2.stats()['Total Trades']}")

In [ ]:
# === CELL 10: Synergy Type 3 - NWRQK → MLMI → FVG ===

@njit
def detect_synergy_type3(nwrqk_bull, nwrqk_bear, mlmi_bull, mlmi_bear,
                        fvg_bull, fvg_bear, n):
    """Detect Type 3 Synergy: NWRQK → MLMI → FVG"""
    long_signals = np.zeros(n, dtype=np.bool_)
    short_signals = np.zeros(n, dtype=np.bool_)
    
    # Window for synergy completion
    window = 30  # bars to complete synergy
    
    for i in range(2, n-window):
        # Check for NWRQK signal
        if nwrqk_bull[i]:  # NWRQK bullish change
            # Look for MLMI confirmation within next bars
            for j in range(i+1, min(i+window//2, n)):
                if mlmi_bull[j]:
                    # Look for FVG confirmation
                    for k in range(j+1, min(i+window, n)):
                        if fvg_bull[k]:
                            long_signals[k] = True
                            break
                    break
        
        elif nwrqk_bear[i]:  # NWRQK bearish change
            # Look for MLMI confirmation within next bars
            for j in range(i+1, min(i+window//2, n)):
                if mlmi_bear[j]:
                    # Look for FVG confirmation
                    for k in range(j+1, min(i+window, n)):
                        if fvg_bear[k]:
                            short_signals[k] = True
                            break
                    break
    
    return long_signals, short_signals

class SynergyType3Strategy:
    """Strategy for Synergy Type 3: NWRQK → MLMI → FVG"""
    
    def __init__(self, data):
        self.data = data
        self.name = "Type 3: NWRQK → MLMI → FVG"
    
    def generate_signals(self):
        """Generate entry and exit signals"""
        print(f"\n🎯 Generating signals for {self.name}...")
        
        # Extract arrays
        nwrqk_bull = self.data['nwrqk_bull_30m'].fillna(False).values
        nwrqk_bear = self.data['nwrqk_bear_30m'].fillna(False).values
        mlmi_bull = self.data['mlmi_bull_30m'].fillna(False).values
        mlmi_bear = self.data['mlmi_bear_30m'].fillna(False).values
        fvg_bull = self.data['FVG_Bull_Active'].values
        fvg_bear = self.data['FVG_Bear_Active'].values
        
        # Detect synergy signals
        long_entries, short_entries = detect_synergy_type3(
            nwrqk_bull, nwrqk_bear, mlmi_bull, mlmi_bear,
            fvg_bull, fvg_bear, len(self.data)
        )
        
        # Generate exit signals
        long_exits = np.zeros(len(self.data), dtype=bool)
        short_exits = np.zeros(len(self.data), dtype=bool)
        
        print(f"  • Long entries: {long_entries.sum():,}")
        print(f"  • Short entries: {short_entries.sum():,}")
        
        return {
            'long_entries': pd.Series(long_entries, index=self.data.index),
            'short_entries': pd.Series(short_entries, index=self.data.index),
            'long_exits': pd.Series(long_exits, index=self.data.index),
            'short_exits': pd.Series(short_exits, index=self.data.index)
        }
    
    def backtest(self):
        """Run vectorbt backtest"""
        signals = self.generate_signals()
        
        # Run backtest with vectorbt
        try:
            portfolio = vbt.Portfolio.from_signals(
                close=self.data['Close'],
                entries=signals['long_entries'] | signals['short_entries'],
                exits=signals['long_exits'] | signals['short_exits'],
                direction=np.where(signals['long_entries'], 1, 
                                 np.where(signals['short_entries'], -1, 0)),
                size=100,
                init_cash=100000,
                fees=0.0001,
                slippage=0.0001,
                freq='5T'
            )
            
            return portfolio
            
        except Exception as e:
            print(f"❌ Backtest error: {e}")
            return None

# Run Type 3 Synergy
if not df_backtest.empty:
    print("="*80)
    print("📊 SYNERGY TYPE 3 - NWRQK → MLMI → FVG")
    print("="*80)
    
    strategy3 = SynergyType3Strategy(df_backtest)
    portfolio3 = strategy3.backtest()
    
    if portfolio3:
        print(f"\n📈 Results:")
        print(f"  • Total Return: {portfolio3.total_return():.2%}")
        print(f"  • Sharpe Ratio: {portfolio3.sharpe_ratio():.2f}")
        print(f"  • Max Drawdown: {portfolio3.max_drawdown():.2%}")
        print(f"  • Total Trades: {portfolio3.stats()['Total Trades']}")

In [ ]:
# === CELL 11: Synergy Type 4 - NWRQK → FVG → MLMI ===

@njit
def detect_synergy_type4(nwrqk_bull, nwrqk_bear, fvg_bull, fvg_bear,
                        mlmi_bull, mlmi_bear, n):
    """Detect Type 4 Synergy: NWRQK → FVG → MLMI"""
    long_signals = np.zeros(n, dtype=np.bool_)
    short_signals = np.zeros(n, dtype=np.bool_)
    
    # Window for synergy completion
    window = 30  # bars to complete synergy
    
    for i in range(2, n-window):
        # Check for NWRQK signal
        if nwrqk_bull[i]:  # NWRQK bullish change
            # Look for FVG within next bars
            for j in range(i+1, min(i+window//2, n)):
                if fvg_bull[j]:
                    # Look for MLMI confirmation
                    for k in range(j+1, min(i+window, n)):
                        if mlmi_bull[k]:
                            long_signals[k] = True
                            break
                    break
        
        elif nwrqk_bear[i]:  # NWRQK bearish change
            # Look for FVG within next bars
            for j in range(i+1, min(i+window//2, n)):
                if fvg_bear[j]:
                    # Look for MLMI confirmation
                    for k in range(j+1, min(i+window, n)):
                        if mlmi_bear[k]:
                            short_signals[k] = True
                            break
                    break
    
    return long_signals, short_signals

class SynergyType4Strategy:
    """Strategy for Synergy Type 4: NWRQK → FVG → MLMI"""
    
    def __init__(self, data):
        self.data = data
        self.name = "Type 4: NWRQK → FVG → MLMI"
    
    def generate_signals(self):
        """Generate entry and exit signals"""
        print(f"\n🎯 Generating signals for {self.name}...")
        
        # Extract arrays
        nwrqk_bull = self.data['nwrqk_bull_30m'].fillna(False).values
        nwrqk_bear = self.data['nwrqk_bear_30m'].fillna(False).values
        fvg_bull = self.data['FVG_Bull_Active'].values
        fvg_bear = self.data['FVG_Bear_Active'].values
        mlmi_bull = self.data['mlmi_bull_30m'].fillna(False).values
        mlmi_bear = self.data['mlmi_bear_30m'].fillna(False).values
        
        # Detect synergy signals
        long_entries, short_entries = detect_synergy_type4(
            nwrqk_bull, nwrqk_bear, fvg_bull, fvg_bear,
            mlmi_bull, mlmi_bear, len(self.data)
        )
        
        # Generate exit signals
        long_exits = np.zeros(len(self.data), dtype=bool)
        short_exits = np.zeros(len(self.data), dtype=bool)
        
        print(f"  • Long entries: {long_entries.sum():,}")
        print(f"  • Short entries: {short_entries.sum():,}")
        
        return {
            'long_entries': pd.Series(long_entries, index=self.data.index),
            'short_entries': pd.Series(short_entries, index=self.data.index),
            'long_exits': pd.Series(long_exits, index=self.data.index),
            'short_exits': pd.Series(short_exits, index=self.data.index)
        }
    
    def backtest(self):
        """Run vectorbt backtest"""
        signals = self.generate_signals()
        
        # Run backtest with vectorbt
        try:
            portfolio = vbt.Portfolio.from_signals(
                close=self.data['Close'],
                entries=signals['long_entries'] | signals['short_entries'],
                exits=signals['long_exits'] | signals['short_exits'],
                direction=np.where(signals['long_entries'], 1, 
                                 np.where(signals['short_entries'], -1, 0)),
                size=100,
                init_cash=100000,
                fees=0.0001,
                slippage=0.0001,
                freq='5T'
            )
            
            return portfolio
            
        except Exception as e:
            print(f"❌ Backtest error: {e}")
            return None

# Run Type 4 Synergy
if not df_backtest.empty:
    print("="*80)
    print("📊 SYNERGY TYPE 4 - NWRQK → FVG → MLMI")
    print("="*80)
    
    strategy4 = SynergyType4Strategy(df_backtest)
    portfolio4 = strategy4.backtest()
    
    if portfolio4:
        print(f"\n📈 Results:")
        print(f"  • Total Return: {portfolio4.total_return():.2%}")
        print(f"  • Sharpe Ratio: {portfolio4.sharpe_ratio():.2f}")
        print(f"  • Max Drawdown: {portfolio4.max_drawdown():.2%}")
        print(f"  • Total Trades: {portfolio4.stats()['Total Trades']}")

In [ ]:
# === CELL 12: Monte Carlo Validation for Each Synergy ===

import matplotlib.pyplot as plt
from scipy import stats

class MonteCarloValidator:
    """Monte Carlo validation for strategy performance"""
    
    def __init__(self, portfolio, strategy_name, n_simulations=1000):
        self.portfolio = portfolio
        self.strategy_name = strategy_name
        self.n_simulations = n_simulations
    
    def run_validation(self):
        """Run Monte Carlo simulation"""
        if not self.portfolio:
            print(f"❌ No portfolio available for {self.strategy_name}")
            return None
        
        print(f"\n🎲 Running Monte Carlo for {self.strategy_name}...")
        
        # Get daily returns
        returns = self.portfolio.returns()
        
        # Original metrics
        original_return = self.portfolio.total_return()
        original_sharpe = self.portfolio.sharpe_ratio()
        
        # Run simulations
        sim_returns = []
        sim_sharpes = []
        
        for _ in range(self.n_simulations):
            # Bootstrap resample
            sampled_returns = np.random.choice(returns, size=len(returns), replace=True)
            
            # Calculate metrics
            total_ret = np.prod(1 + sampled_returns) - 1
            sharpe = np.mean(sampled_returns) / np.std(sampled_returns) * np.sqrt(252) if np.std(sampled_returns) > 0 else 0
            
            sim_returns.append(total_ret)
            sim_sharpes.append(sharpe)
        
        # Calculate percentiles
        return_percentile = stats.percentileofscore(sim_returns, original_return)
        sharpe_percentile = stats.percentileofscore(sim_sharpes, original_sharpe)
        
        print(f"  • Return Percentile: {return_percentile:.1f}%")
        print(f"  • Sharpe Percentile: {sharpe_percentile:.1f}%")
        
        return {
            'original_return': original_return,
            'original_sharpe': original_sharpe,
            'return_percentile': return_percentile,
            'sharpe_percentile': sharpe_percentile,
            'sim_returns': sim_returns,
            'sim_sharpes': sim_sharpes
        }

# Run Monte Carlo for each synergy
print("="*80)
print("🎲 MONTE CARLO VALIDATION FOR ALL SYNERGIES")
print("="*80)

# Validate each synergy if portfolio exists
synergy_results = {}

if 'portfolio1' in locals() and portfolio1:
    validator1 = MonteCarloValidator(portfolio1, "Type 1: MLMI → FVG → NWRQK")
    synergy_results['Type 1'] = validator1.run_validation()

if 'portfolio2' in locals() and portfolio2:
    validator2 = MonteCarloValidator(portfolio2, "Type 2: MLMI → NWRQK → FVG")
    synergy_results['Type 2'] = validator2.run_validation()

if 'portfolio3' in locals() and portfolio3:
    validator3 = MonteCarloValidator(portfolio3, "Type 3: NWRQK → MLMI → FVG")
    synergy_results['Type 3'] = validator3.run_validation()

if 'portfolio4' in locals() and portfolio4:
    validator4 = MonteCarloValidator(portfolio4, "Type 4: NWRQK → FVG → MLMI")
    synergy_results['Type 4'] = validator4.run_validation()

# Summary
print("\n" + "="*80)
print("📊 MONTE CARLO SUMMARY")
print("="*80)

for synergy_name, results in synergy_results.items():
    if results:
        avg_percentile = (results['return_percentile'] + results['sharpe_percentile']) / 2
        print(f"\n{synergy_name}:")
        print(f"  • Average Percentile: {avg_percentile:.1f}%")
        
        if avg_percentile > 80:
            print("  • Rating: ✅ EXCELLENT")
        elif avg_percentile > 60:
            print("  • Rating: 👍 GOOD")
        else:
            print("  • Rating: ⚠️ NEEDS IMPROVEMENT")

In [ ]:
# === CELL 13: Final Summary and Recommendations ===

print("="*80)
print("🏁 ALGOSPACE STRATEGY - FINAL SUMMARY")
print("="*80)

# Collect all results
all_portfolios = {
    'Type 1: MLMI → FVG → NWRQK': portfolio1 if 'portfolio1' in locals() else None,
    'Type 2: MLMI → NWRQK → FVG': portfolio2 if 'portfolio2' in locals() else None,
    'Type 3: NWRQK → MLMI → FVG': portfolio3 if 'portfolio3' in locals() else None,
    'Type 4: NWRQK → FVG → MLMI': portfolio4 if 'portfolio4' in locals() else None
}

# Performance comparison
print("\n📊 PERFORMANCE COMPARISON:")
print("-" * 80)
print(f"{'Synergy Type':<30} {'Return':>10} {'Sharpe':>10} {'Max DD':>10} {'Trades':>10}")
print("-" * 80)

best_return = -float('inf')
best_sharpe = -float('inf')
best_synergy_return = None
best_synergy_sharpe = None

for name, portfolio in all_portfolios.items():
    if portfolio:
        ret = portfolio.total_return()
        sharpe = portfolio.sharpe_ratio()
        dd = portfolio.max_drawdown()
        trades = portfolio.stats()['Total Trades']
        
        print(f"{name:<30} {ret:>10.2%} {sharpe:>10.2f} {dd:>10.2%} {trades:>10}")
        
        if ret > best_return:
            best_return = ret
            best_synergy_return = name
        
        if sharpe > best_sharpe:
            best_sharpe = sharpe
            best_synergy_sharpe = name

print("-" * 80)

# Recommendations
print("\n🎯 RECOMMENDATIONS:")
if best_synergy_return:
    print(f"  • Best Return: {best_synergy_return} ({best_return:.2%})")
if best_synergy_sharpe:
    print(f"  • Best Risk-Adjusted: {best_synergy_sharpe} (Sharpe: {best_sharpe:.2f})")

print("\n💡 STRATEGY INSIGHTS:")
print("  • Each synergy type captures different market dynamics")
print("  • Consider using multiple synergies with position sizing")
print("  • Monitor performance and adjust parameters as needed")

print("\n✅ STRATEGY ANALYSIS COMPLETE!")
print("="*80)